# 🦜🔗 LangChain 기반 RAG 실습 (2025년 최신)

이번 실습에서는 **최신 LangChain API**를 활용하여 RAG(Retrieval-Augmented Generation) 시스템을 구축해보겠습니다.

> 📢 **OpenAI RAG와 LangChain RAG의 차이점**
> 
> - OpenAI는 `vector_stores` API로 간단하게 RAG를 구현할 수 있지만, **커스터마이징에 제한**이 있습니다.
> - LangChain은 **Document Loader, Text Splitter, Embedding, Vector Store, Retriever** 등 각 컴포넌트를 모듈화하여 **유연한 파이프라인 구성**이 가능합니다.
> - LangChain의 최신 `create_agent` API와 `@tool` 데코레이터를 활용하면 **Agentic RAG**도 손쉽게 구현할 수 있습니다.


## 📋 목차

1. [환경 설정](#1-환경-설정)
2. [RAG 아키텍처 이해하기](#2-rag-아키텍처-이해하기)
3. [문서 로딩 (Document Loading)](#3-문서-로딩-document-loading)
4. [텍스트 분할 (Text Splitting)](#4-텍스트-분할-text-splitting)
5. [임베딩 및 벡터 저장소 (Embeddings & Vector Store)](#5-임베딩-및-벡터-저장소)
6. [2-Step RAG 구현](#6-2-step-rag-구현)
7. [RAG Agent 구현](#7-rag-agent-구현)
8. [실습: 질의응답 테스트](#8-실습-질의응답-테스트)


---
## 1. 환경 설정

먼저 필요한 패키지들을 설치하고 환경을 구성해보겠습니다.


In [ ]:
# 필요한 패키지 설치
%pip install -qU langchain langchain-openai langchain-community langchain-text-splitters pypdf python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키가 설정되었는지 확인
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API 키가 설정되었습니다.")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인해주세요.")


---
## 2. RAG 아키텍처 이해하기

### RAG의 핵심 개념

LLM은 강력하지만 두 가지 주요 한계가 있습니다:

1. **유한한 컨텍스트** - 전체 문서를 한 번에 처리할 수 없습니다
2. **정적인 지식** - 학습 데이터가 특정 시점에 고정되어 있습니다

RAG는 **쿼리 시점에 관련 외부 지식을 검색**하여 이 문제를 해결합니다.

### Langchain이 소개하는 RAG 아키텍처 종류

| 아키텍처 | 설명 | 제어 | 유연성 | 지연 시간 | 사용 사례 |
|---------|------|------|--------|----------|----------|
| **2-Step RAG** | 검색 후 생성. 단순하고 예측 가능 | ✅ 높음 | ❌ 낮음 | ⚡ 빠름 | FAQ, 문서 봇 |
| **Agentic RAG** | LLM 에이전트가 검색 여부를 결정 | ❌ 낮음 | ✅ 높음 | ⏳ 가변 | 다중 도구 연구 보조 |
| **Hybrid** | 두 접근법의 조합, 검증 단계 포함 | ⚖️ 중간 | ⚖️ 중간 | ⏳ 가변 | 품질 검증이 필요한 Q&A |

이번 실습에서는 **2-Step RAG**와 **Agentic RAG** 모두 구현해보겠습니다.


---
## 3. 문서 로딩 (Document Loading)

LangChain은 다양한 소스(Google Drive, Slack, Notion, PDF 등)에서 데이터를 로드할 수 있는 Document Loader를 제공합니다.

여기서는 `PyPDFLoader`를 사용하여 PDF 문서를 로드해보겠습니다.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 설정 (OpenAI RAG 실습과 동일한 문서 사용)
file_path = "docs/DeepSeek_OCR_paper.pdf"

# PDF 로더 초기화 및 문서 로드
loader = PyPDFLoader(file_path)
docs = loader.load()

print(f"📄 로드된 문서 수: {len(docs)} 페이지")


In [ ]:
# 첫 번째 페이지 내용 확인
print(f"📝 첫 번째 페이지 내용 (처음 500자):\n")
print(docs[0].page_content[:500])
print("\n" + "="*50)
print(f"\n📋 메타데이터: {docs[0].metadata}")


---
## 4. 텍스트 분할 (Text Splitting)

로드된 문서가 너무 길면 모델의 컨텍스트 윈도우에 맞지 않을 수 있습니다. 

`RecursiveCharacterTextSplitter`를 사용하여 문서를 적절한 크기의 청크로 나눠보겠습니다.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
# - chunk_size: 각 청크의 최대 문자 수
# - chunk_overlap: 청크 간 겹치는 문자 수 (문맥 유지를 위해)
# - add_start_index: 원본 문서에서의 시작 위치 추적

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 청크 크기 (문자 수)
    chunk_overlap=200,    # 청크 간 겹침 (문자 수)
    add_start_index=True  # 원본 문서에서의 인덱스 추적
)

# 문서 분할
all_splits = text_splitter.split_documents(docs)

print(f"✂️ 원본 문서 {len(docs)}페이지가 {len(all_splits)}개의 청크로 분할되었습니다.")


In [ ]:
# 분할된 청크 샘플 확인
print("📦 첫 번째 청크:")
print(f"내용: {all_splits[0].page_content[:300]}...")
print(f"\n메타데이터: {all_splits[0].metadata}")
print(f"\n청크 길이: {len(all_splits[0].page_content)} 문자")


### 💡 왜 텍스트를 분할해야 할까요?

1. **검색 정확도 향상**: 작은 청크에서 관련 정보를 더 정확하게 찾을 수 있습니다
2. **컨텍스트 윈도우 제한**: LLM은 한 번에 처리할 수 있는 토큰 수에 제한이 있습니다
3. **비용 효율성**: 전체 문서 대신 관련 청크만 전달하여 API 비용을 절약합니다

`chunk_overlap`을 설정하는 이유는 **문맥이 청크 경계에서 끊기는 것을 방지**하기 위함입니다.


---
## 5. 임베딩 및 벡터 저장소

분할된 청크를 벡터로 변환하고 벡터 저장소에 저장합니다.

### 임베딩(Embedding)이란?

텍스트를 **의미적 유사도를 계산할 수 있는 수치 벡터**로 변환하는 과정입니다. 비슷한 의미의 텍스트는 벡터 공간에서 가까이 위치하게 됩니다.


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 인메모리 벡터 저장소 생성 및 문서 인덱싱
vector_store = InMemoryVectorStore.from_documents(
    documents=all_splits,
    embedding=embeddings
)

print("✅ 벡터 저장소 생성 완료!")


In [ ]:
# 유사도 검색 테스트
query = "DeepSeek-OCR 모델의 주요 특징은 무엇인가요?"

# 상위 3개의 유사한 문서 검색
similar_docs = vector_store.similarity_search(query, k=3)

print(f"🔍 검색 쿼리: {query}\n")
for i, doc in enumerate(similar_docs, 1):
    print(f"[결과 {i}]")
    print(f"📄 페이지: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 내용: {doc.page_content[:200]}...\n")


---
## 6. 2-Step RAG 구현

가장 기본적인 RAG 아키텍처입니다. **검색 → 생성** 두 단계로 작동합니다.

```
사용자 질문 → 관련 문서 검색 → LLM으로 답변 생성 → 사용자에게 반환
```


In [ ]:
from langchain_openai import ChatOpenAI

# 채팅 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Retriever 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print("✅ 2-Step RAG 구성 요소 준비 완료!")


In [ ]:
def rag_2step(question: str) -> dict:
    """2-Step RAG 구현: 검색 후 생성"""
    
    # Step 1: 관련 문서 검색
    retrieved_docs = retriever.invoke(question)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    
    # Step 2: LLM으로 답변 생성
    system_prompt = f"""You are a helpful assistant who is good at analyzing source information and answering questions.
Use the following source documents to answer the user's questions.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Answer in Korean.

Documents:
{docs_content}"""
    
    response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ])
    
    return {
        "question": question,
        "answer": response.content,
        "source_documents": retrieved_docs
    }


In [ ]:
# 2-Step RAG 테스트
question = "DeepSeek-OCR 모델의 주요 특징은 무엇인가요?"
result = rag_2step(question)

print(f"❓ 질문: {result['question']}\n")
print(f"💡 답변: {result['answer']}\n")
print(f"📚 참조 문서 수: {len(result['source_documents'])}개")


---
## 7. RAG Agent 구현

이제 최신 LangChain API인 `create_agent`와 `@tool` 데코레이터를 사용하여 **Agentic RAG**를 구현해보겠습니다.

### Agentic RAG란?

- LLM 에이전트가 **검색 여부와 방법을 스스로 결정**합니다
- 복잡한 질문에 대해 **여러 번의 검색**을 수행할 수 있습니다
- **다양한 도구(Tools)**를 활용할 수 있습니다


In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from langsmith import traceable

# @tool 데코레이터를 사용하여 검색 도구 정의
# response_format="content_and_artifact"로 설정하면 문서 메타데이터도 함께 반환됩니다

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """DeepSeek-OCR 논문에서 관련 정보를 검색합니다.
    
    Args:
        query: 검색할 내용을 설명하는 쿼리 문자열
    
    Returns:
        검색된 문서의 내용과 메타데이터
    """
    retrieved_docs = retriever.invoke(query)
    
    # 문서 내용을 문자열로 결합
    content = "\n\n---\n\n".join(
        f"[페이지 {doc.metadata.get('page', 'N/A')}]\n{doc.page_content}" 
        for doc in retrieved_docs
    )
    
    # artifact에 원본 문서 메타데이터 저장
    artifact = [
        {"page": doc.metadata.get('page'), "source": doc.metadata.get('source')}
        for doc in retrieved_docs
    ]
    
    return content, artifact

print("✅ 검색 도구가 정의되었습니다.")


In [ ]:
# RAG 에이전트 생성
tools = [retrieve_context]

system_prompt = """You are a helpful assistant specialized in analyzing academic papers.
You have access to a tool that retrieves context from the DeepSeek-OCR paper.
Use the tool whenever you need specific information from the paper.

Guidelines:
- Always use the retrieve_context tool to find relevant information before answering
- If the first search doesn't provide enough information, try different search queries
- Answer in Korean
"""

rag_agent = create_agent(
    model="gpt-4.1-mini",
    tools=tools,
    system_prompt=system_prompt
)

print("✅ RAG 에이전트가 생성되었습니다.")


In [ ]:
# RAG 에이전트 테스트 - 단일 검색이 필요한 질문
question = "DeepSeek-OCR의 압축률(compression ratio)은 얼마인가요?"

print(f"❓ 질문: {question}\n")
print("🤖 에이전트 응답 과정:\n")

for event in rag_agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()


In [ ]:
# RAG 에이전트 테스트 - 복합 검색이 필요한 질문
complex_question = """DeepSeek-OCR의 아키텍처를 설명해주세요.
그리고 OmniDocBench에서의 성능은 어떤가요?"""

print(f"❓ 복합 질문: {complex_question}\n")
print("🤖 에이전트 응답 과정:\n")

for event in rag_agent.stream(
    {"messages": [{"role": "user", "content": complex_question}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()


### 💡 Agentic RAG의 장점

1. **동적 검색**: 질문의 복잡도에 따라 검색 횟수를 조절합니다
2. **다중 쿼리**: 하나의 질문에서 여러 검색 쿼리를 생성할 수 있습니다
3. **도구 확장**: 웹 검색, 계산기 등 다른 도구와 결합할 수 있습니다


---
### 📚 참고 자료

- [LangChain Retrieval Documentation](https://docs.langchain.com/oss/python/langchain/retrieval)
- [LangChain RAG Tutorial](https://docs.langchain.com/oss/python/langchain/rag)
- [LangGraph Agentic RAG](https://docs.langchain.com/oss/python/langgraph/agentic-rag)
